# Test Locally Built InterpretML Library

This notebook is designed to help you test the `interpret` library after building it locally with your custom changes (e.g., a new C++ loss function).

**Prerequisites:**
1.  You have successfully run `make build` and `make install` from the `/home/diego/Dropbox/DropboxGit/interpret/scripts/` directory.
2.  You are running this notebook in the same Python environment where the local `interpret` package was installed.

In [1]:
# Import the interpret library and check version
try:
    import interpret
    from interpret import show # For visualizing explanations
    # Attempt to import a core component, e.g., an explainer or data utility
    from interpret.glassbox import ExplainableBoostingClassifier 
    from interpret.data import ClassHistogram # A data utility
    
    print("InterpretML library imported successfully.")
    print(f"InterpretML version: {interpret.__version__}")
    # You can also check the path to see if it's from your local repository
    print(f"InterpretML path: {interpret.__path__}")

except ImportError as e:
    print(f"Error importing InterpretML: {e}")
    print("Please ensure you have correctly run 'make build' and 'make install' from the 'scripts' directory.")
    print("Also, verify that this notebook is using the Python environment where the package was installed.")
except Exception as e:
    print(f"An unexpected error occurred during import: {e}")

InterpretML library imported successfully.
InterpretML version: 0.6.10
InterpretML path: ['/home/diego/Dropbox/DropboxGit/interpret/python/interpret-core/interpret']


## Basic Functionality Test

Let's try to use a common component like the Explainable Boosting Machine (EBM) to see if the library loads and executes basic operations. If your C++ changes affect EBM or related core components, this is a good place to start.

In [2]:
# Example: Using Explainable Boosting Machine (EBM)
# This model often has performance-critical parts that might be implemented in C++.

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import pandas as pd

# Generate some synthetic classification data
X, y = make_classification(n_samples=200, n_features=5, random_state=42)
feature_names = [f"feature_{i}" for i in range(X.shape[1])]
X_df = pd.DataFrame(X, columns=feature_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

print("Data prepared for EBM.")

try:
    # Initialize and train an EBM
    # If your new loss is for EBM, you might need to specify it here if applicable
    ebm = ExplainableBoostingClassifier(random_state=42, feature_names=feature_names)
    ebm.fit(X_train, y_train)
    print("EBM model trained successfully.")

    # Get global explanations
    ebm_global = ebm.explain_global()
    print("\nGlobal Explanations obtained from EBM.")
    # In a full Jupyter environment, you can visualize this:
    # show(ebm_global)

    # Get local explanations for a few instances
    ebm_local = ebm.explain_local(X_test.head(), y_test[:len(X_test.head())])
    print("\nLocal Explanations for the first few test instances obtained.")
    # In a full Jupyter environment, you can visualize this:
    # show(ebm_local)

    print("\nBasic EBM test complete.")
    print("If your changes are related to EBM, observe its behavior and outputs carefully.")

except Exception as e:
    print(f"An error occurred while testing EBM: {e}")
    print("This could indicate an issue with the compiled components or your changes.")

Data prepared for EBM.
EBM model trained successfully.

Global Explanations obtained from EBM.

Local Explanations for the first few test instances obtained.

Basic EBM test complete.
If your changes are related to EBM, observe its behavior and outputs carefully.


## Test Your Specific Changes

Now, add cells below to specifically test the new loss function or other C++ modifications you have implemented.

For example, if you added a new loss function `my_custom_loss` to a specific model:
1.  Initialize the model with `loss='my_custom_loss'`.
2.  Train it on appropriate data.
3.  Evaluate its performance and behavior (e.g., convergence, explanation stability, comparison with existing losses).
4.  Check for any errors or unexpected outputs.

In [ ]:
# Test the custom "negative_binomial" objective

from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd

print("Attempting to test the 'negative_binomial' objective...")

# Generate synthetic regression data
# For Negative Binomial with a log link, y should be positive counts.
# We'll generate general regression data and transform y to be positive.
X_reg, y_reg = make_regression(n_samples=100, n_features=3, random_state=42, noise=0.1)

# Ensure y_reg is positive for the log link used by negative_binomial
# (actual negative binomial loss expects counts, but this is a structural test)
y_reg_positive = np.abs(y_reg) + 1.0 

feature_names_reg = [f"reg_feature_{i}" for i in range(X_reg.shape[1])]
X_reg_df = pd.DataFrame(X_reg, columns=feature_names_reg)

print(f"\nGenerated regression data. X_reg_df.shape: {X_reg_df.shape}, y_reg_positive.shape: {y_reg_positive.shape}")
print(f"Sample y_reg_positive values: {y_reg_positive[:5]}")

try:
    # Initialize ExplainableBoostingRegressor with the custom objective
    # The 'negative_binomial' objective is registered to take an 'alpha' parameter.
    ebm_nb = ExplainableBoostingRegressor(
        objective="negative_binomial",
        # objective_params are passed directly to the C++ constructor
        # For "negative_binomial", we registered it with an "alpha" parameter.
        objective_params={"alpha": 1.5}, # Example alpha value
        feature_names=feature_names_reg,
        random_state=42
    )
    print(f"\nInitialized ExplainableBoostingRegressor with objective='negative_binomial' and alpha=1.5")

    # Fit the model
    ebm_nb.fit(X_reg_df, y_reg_positive)
    print("\nSuccessfully trained EBM with 'negative_binomial' objective.")
    print("This indicates the objective was found and the model training process initiated without crashing.")
    print("Further tests should validate the mathematical correctness of the loss, gradient, and hessian if they were implemented.")

    # Optionally, you can try to get explanations if the model trained
    # ebm_nb_global = ebm_nb.explain_global()
    # print("\nGlobal explanations obtained.")
    # from interpret import show
    # show(ebm_nb_global)

except Exception as e:
    print(f"\nAn error occurred while testing the 'negative_binomial' objective: {e}")
    print("Check the following:")
    print("1. The 'negative_binomial' objective is correctly registered in 'objective_registrations.hpp'.")
    print("2. The 'NegativeBinomialObjective.hpp' class constructor matches the registration parameters (e.g., takes 'alpha').")
    print("3. The library was successfully rebuilt and reinstalled ('make build' and 'make install').")
    print("4. The Python environment for this notebook is the one where the local build was installed.")